In [70]:
import pandas as pd
import numpy as np
import copy

In [71]:
# загрузка данных

df = pd.read_excel('data_with_score_directions_roles.xlsx')
projects_df = pd.read_excel('projects_raw_data.xlsx')

In [72]:
df.columns

Index(['Отметка времени', 'Единая учетная запись СПбГУ (например, ST000000)',
       'Выберете интересующие Вас роли, в  IT [Бизнес и системный анализ]',
       'Выберете интересующие Вас роли, в  IT [Дизайн UX / UI и проектирование интерфейсов]',
       'Выберете интересующие Вас роли, в  IT [Анализ и инженерия данных]',
       'Выберете интересующие Вас роли, в  IT [Backend - разработка]',
       'Выберете интересующие Вас роли, в  IT [Frontend - разработка]',
       'Выберете интересующие Вас роли, в  IT [Тестирование и обеспечение качества]',
       'Выберете интересующие Вас роли, в  IT [Развертывание и внедрение]',
       'Выберете интересующие Вас роли, в  IT [Управление командой / проектом]',
       'Выберете интересные для вас направление проекта (в направлении проекта могут быть все роли, указанные выше) [Разработка веб-сервисов (сайты, сервисы)]',
       'Выберете интересные для вас направление проекта (в направлении проекта могут быть все роли, указанные выше) [Анализ текст

In [73]:
df

,Отметка времени,"Единая учетная запись СПбГУ (например, ST000000)","Выберете интересующие Вас роли, в IT [Бизнес и системный анализ]","Выберете интересующие Вас роли, в IT [Дизайн UX / UI и проектирование интерфейсов]","Выберете интересующие Вас роли, в IT [Анализ и инженерия данных]","Выберете интересующие Вас роли, в IT [Backend - разработка]","Выберете интересующие Вас роли, в IT [Frontend - разработка]","Выберете интересующие Вас роли, в IT [Тестирование и обеспечение качества]","Выберете интересующие Вас роли, в IT [Развертывание и внедрение]","Выберете интересующие Вас роли, в IT [Управление командой / проектом]",...,"Что Вам поможет развиваться быстрее в том направлении деятельности, которое Вы выбрали?",Year,Season,score,hard_skills_score,soft_skills_score,analytics_skills_score,knowladges_skills_score,directions,roles
0,2023-10-07 16:02:10.676,ST110311,3,1,1.0,1,1,3,1,3,...,NaN,2023,Осень,10.1,1.2,4.6,3.3,1.0,"['Web', 'NLP']",[]
1,2023-10-02 16:23:02.340,ST063118,1,3,1.0,5,3,3,3,3,...,"Опытные наставники, интересные задачи",2023,Осень,13.7,2.2,3.4,4.2,3.9,"['Time Series', 'DS', 'Robotics']",['backend developer']
2,2023-09-15 19:00:59.596,ST114946,1,3,1.0,3,5,3,1,1,...,Чёткий план действий,2023,Осень,13.7,2.5,3.6,4.5,3.1,"['CV', 'DS']",['frontend developer']
3,2023-09-11 14:35:19.227,ST112047,3,3,3.0,1,1,3,1,5,...,Прежний опыт в написанию проектов и любовь к р...,2023,Осень,10.8,1.0,5.0,3.7,1.1,"['Web', 'NLP', 'Robotics']",['pm']
4,2023-09-10 22:04:15.396,ST112471,1,1,1.0,1,1,3,1,5,...,Личный рост,2023,Осень,11.3,0.0,4.6,4.8,1.9,[],['pm']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2019-08-26 11:39:44.782,ST061362,4,5,NaN,1,1,2,2,5,...,Время для развития навыков\nВозможность общать...,2019,Осень,11.4,2.6,2.4,3.5,2.9,"['Time Series', 'DS', 'Design']","['system or business analyst', 'ux/ui', 'pm']"
345,2019-08-26 11:38:34.578,ST061177,1,5,NaN,4,4,1,2,2,...,NaN,2019,Осень,7.0,1.7,1.8,1.7,1.8,"['Web', 'NLP', 'Time Series', 'DS']","['ux/ui', 'backend developer', 'frontend devel..."
346,2019-08-26 11:37:09.651,ST054461,1,5,NaN,4,4,2,2,2,...,Чётко поставленные задачи,2019,Осень,11.0,1.8,2.2,4.0,3.0,"['NLP', 'Time Series', 'DS', 'Robotics']","['ux/ui', 'backend developer', 'frontend devel..."
347,2019-08-26 11:36:51.336,ST063110,1,2,NaN,4,4,4,2,1,...,"Считаю, что интересная, хорошо выстроенная задача",2019,Осень,9.1,2.0,1.6,2.7,2.8,"['Web', 'NLP', 'Time Series', 'DS', 'Robotics'...","['backend developer', 'frontend developer', 'qa']"


In [74]:
def df_projects(df):
   projects = []
   df = df.reset_index()
   for _, project_df in df.iterrows():
      project = dict()
      project['name'] = project_df['Проект']
      project['year'] = project_df['Год']
      project['season'] = project_df['Сезон']
      d = []
      for w in project_df['Направление'].split(sep=','):
         d.append(w.strip())
      project['directions'] = d
      c=[]
      for w in project_df['Состав команды'].split(sep=','):
         if ':' in w:
            student_st = w.split(sep=':')[1]
            role =  w.split(sep=':')[0]
            project[role.strip().lower()] = student_st.strip()
            c.append(role.strip().lower())
         else:   
            project[w.strip().lower()] = ''
            c.append(w.strip().lower())
      project['compound'] = c
      
      projects.append(project)
   return projects

def df_students(df):
   students = []
   df = df.reset_index()

   for _, student_df in df.iterrows():
      student = dict()
      student['st'] = student_df['Единая учетная запись СПбГУ (например, ST000000)']
      student['year'] = student_df['Year']
      student['score'] = student_df['score']
      student['directions'] = eval(student_df['directions'])
      student['roles'] = eval(student_df['roles'])

      students.append(student)

   return students


In [75]:
# add version with pre-selected students

def data_preparation(df_students, df_projects):
   students = []
   students_df = df_students.reset_index()

   for _, student_df in students_df.iterrows():
      student = dict()
      student['st'] = student_df['Единая учетная запись СПбГУ (например, ST000000)']
      student['year'] = student_df['Year']
      student['season'] =  student_df['Season']
      student['score'] = student_df['score']
      student['directions'] = eval(student_df['directions'])
      student['roles'] = eval(student_df['roles'])
      students.append(student)
    
   projects = []
   projects_df = df_projects.reset_index()
   for _, project_df in projects_df.iterrows():
      project = dict()
      project['name'] = project_df['Проект']
      project['year'] = project_df['Год']
      project['season'] = project_df['Сезон']
      d = []
      for w in project_df['Направление'].split(sep=','):
         d.append(w.strip())
      project['directions'] = d
      c=[]
      for w in project_df['Состав команды'].split(sep=','):
         if ':' in w:
            project['team'] = []
            student_st = w.split(sep=':')[1].strip()
            for s in students:
               if s['st'] == student_st:
                  s['assigned'] = True
                  project['team'].append(s)
                  role = w.split(sep=':')[0]
                  project[role.strip().lower()] = []
                  project[role.strip().lower()].append([s['st'], s['score'], s['year'], s['season']])
                  break
            c.append(role.strip().lower())
         else:   
            project[w.strip().lower()] = []
            c.append(w.strip().lower())
      project['compound'] = c
      
      projects.append(project)

   return students, projects


In [76]:
def student_exchange(project1, student_st1, project2, student_st2, distprojects):
  distributed_projects_new = copy.deepcopy(distprojects) 
  student1 = []
  student2 = []
  p1_index = 0
  p2_index = 0
  student1_role = '' 
  student2_role = ''
  i = 0
  for project in distributed_projects_new:
    if project['name'] == project1:
      p1_index = distributed_projects_new.index(project)
      p_1 = project
      for student in project['team']:
        if student.split(' ')[0] == student_st1:
          student1 = student
          project['team'].remove(student)
          for role in project['compound']:
            if student1 in project[role]:
               project[role].remove(student1)
               student1_role = role
               break
          break
      if len(student1) == 0: return -1
      i += 1
    
    if project['name'] == project2:
      p2_index = distributed_projects_new.index(project)
      p_2 = project
      for student in project['team']:     
        if student.split(' ')[0] == student_st2:
          student2 = student
          project['team'].remove(student)
          for role in project['compound']:
            if student2 in project[role]:
               project[role].remove(student2)
               student2_role = role
               break
          break
      if len(student2) == 0: return -1  
      i += 1
    
    if i == 2:
      break

  if (p1_index == 0) | (p2_index == 0): return -2

  if len(student1_role) != 0 and len(student2) != 0:
     p_1['team'].append(student2)
     p_1[student1_role].append([student2[0], student2[1], student2[2]])
     distributed_projects_new[p1_index] = p_1
  elif len(student1_role) == 0 and len(student2) != 0:
    p_1['team'].append(student2)
    distributed_projects_new[p1_index] = p_1   
    
  if len(student2_role) != 0 and len(student1) != 0:
     p_2['team'].append(student1)
     p_2[student2_role].append([student1[0], student1[1], student1[2]]) 
     distributed_projects_new[p2_index] = p_2
  elif len(student2_role) == 0 and len(student1) != 0:
     p_2['team'].append(student1) 
     distributed_projects_new[p2_index] = p_2
     
  return distributed_projects_new

In [77]:
def add_new_student(distributed_projects, student=None, TheProject = None):
    if student is None:
        return -1
    else:
        for st in student: 
            if TheProject is not None:
                for project in distributed_projects:
                    if project['name'] == TheProject:
                        project['team'].append(st)
                        return distributed_projects
            else:
                candidates = [p for p in distributed_projects if len(set(p['directions']).intersection(st['directions'])) > 0 and p['status'] == 'incomplete']
                if len(candidates) != 0:
                    looser = candidates[0]
                    index = 0
                    for p in candidates:
                        if p['team_mean_score'] < looser['team_mean_score']:
                            looser = p
                            index = candidates.index(p)
                    candidates[index]['team'].append([st['st'], st['score'], st['year'], st['season']])
                    n = len(candidates[index]['team'])
                    candidates[index]['team_mean_score'] = (candidates[index]['team_mean_score'] * (n - 1) + st['score']) / n 
                    candidates[index]['remainder'] -= 1   
                    if candidates[index]['remainder'] == 0:    
                        candidates[index]['status'] = 'complete'
                    return candidates        
                else:
                    looser = distributed_projects[0]
                    index = 0
                    for p in distributed_projects:
                        if p['team_mean_score'] < looser['team_mean_score']:
                            looser = p
                            index = distributed_projects.index(p)
                    distributed_projects[index]['team'].append([st['st'], st['score'], st['year'], st['season']])
                    n = len(distributed_projects[index]['team'])
                    distributed_projects[index]['team_mean_score'] = (distributed_projects[index]['team_mean_score'] * (n - 1) + st['score']) / n 
                    distributed_projects[index]['remainder'] -= 1   
                    if distributed_projects[index]['remainder'] == 0:    
                        distributed_projects[index]['status'] = 'complete'
                    return distributed_projects   

In [78]:
def get_roles (students):
   roles = []
   for student in students:
      for role in student['roles']:
         roles.append(role.lower())
   roles = set(roles)
   return roles


In [79]:
def get_directions_of_projects (projects):
   directions = []
   for project in projects:
      for direction in project['directions']:
         directions.append(direction.lower())
   directions = set(directions)
   return directions


In [80]:
def create_dict_of_students_by_direction (directions, students):
   directions_students={direction: [] for direction in directions}
   for student in students:
      for direction in student['directions']:
         if direction.lower() in directions:
            directions_students[direction.lower()].append(student) 
   return directions_students


In [81]:
def avg_score (students):
   scores = [student["score"] for student in students]
   if len(scores) == 0:
        return None
   return sum(scores)/len(scores)


In [82]:
# v1 жадный подход: сначала в команду добавляется студент с лучшим score
'''
def distribute_students_1 (directions, students, projects):
   direction_students  = create_dict_of_students_by_direction(directions, students)
   for project in projects:
      project['team'] = []
      candidates = []
      for direction in project['direction']:
         for student in direction_students[direction.lower()]: candidates.append(student)
      avg = avg_score(candidates)
      for candidate in candidates:
         avg_proj = candidate['score'] / avg
         candidate['avg_proj'] = avg_proj

# Сортировка кандидатов по их avg_proj (в порядке убывания, первым идет более высокий средний балл)
      candidates.sort(key=lambda x: x['avg_proj'], reverse=True)

# Распределяйте студентов по проекту до тех пор, пока размер команды не будет достигнут
      while project['team_size'] > 0 and len(candidates) > 0:
         student = candidates.pop(0)  
         project['team'].append(student)
         project['team_size'] -= 1

      print(f"Project: {project['name']}, {project['direction']}")
      print("Assigned Students:")
      for student in project['team']:
          print(f"- {student['name']}, {student['score']}")
      print()'''      

'\ndef distribute_students_1 (directions, students, projects):\n   direction_students  = create_dict_of_students_by_direction(directions, students)\n   for project in projects:\n      project[\'team\'] = []\n      candidates = []\n      for direction in project[\'direction\']:\n         for student in direction_students[direction.lower()]: candidates.append(student)\n      avg = avg_score(candidates)\n      for candidate in candidates:\n         avg_proj = candidate[\'score\'] / avg\n         candidate[\'avg_proj\'] = avg_proj\n\n# Сортировка кандидатов по их avg_proj (в порядке убывания, первым идет более высокий средний балл)\n      candidates.sort(key=lambda x: x[\'avg_proj\'], reverse=True)\n\n# Распределяйте студентов по проекту до тех пор, пока размер команды не будет достигнут\n      while project[\'team_size\'] > 0 and len(candidates) > 0:\n         student = candidates.pop(0)  \n         project[\'team\'].append(student)\n         project[\'team_size\'] -= 1\n\n      print(f"P

In [83]:
# v2
'''def distribute_students_2(directions, students, projects):
    direction_students = create_dict_of_students_by_direction(directions, students)
    avg = avg_score(students)

    for project in projects:
        project['team'] = []
        candidates = []

        for direction in project['direction']:
            candidates.extend(direction_students[direction.lower()])

        for candidate in candidates:
            candidate['avg_proj'] = candidate['score'] / avg  

        # Сортировка кандидатов внутри данного направления по avg_proj (descending order)
        candidates.sort(key=lambda x: x['avg_proj'])

        while project['team_size'] > 0 and len(candidates) > 0:
            student = candidates.pop(0)  # Получить лучшего студента (с наивысшим avg_proj)
            project['team'].append(student)
            project['team_size'] -= 1

        # Пересчитать средний балл для команды внутри этого проекта
        team_avg_score = avg_score(project['team'])

        # Если команда меньше заданного размера, добавьте других студентов из направления
        while project['team_size'] > 0 and len(candidates) > 0:
            remaining_students = [student for student in candidates if student['direction'].lower() in project['direction']]
            
            if remaining_students:
                # Сортировать оставшихся студентов по разнице между их баллами и средним баллом команды
                remaining_students.sort(key=lambda x: abs(x['score'] - team_avg_score))
                student = remaining_students.pop(0)
                project['team'].append(student)
                project['team_size'] -= 1

    for project in projects:
        print(f"Project: {project['name']}, {project['direction']}")
        print("Assigned Students:")
        for student in project['team']:
            print(f"- {student['name']}, {student['score']}")
        print()'''

'def distribute_students_2(directions, students, projects):\n    direction_students = create_dict_of_students_by_direction(directions, students)\n    avg = avg_score(students)\n\n    for project in projects:\n        project[\'team\'] = []\n        candidates = []\n\n        for direction in project[\'direction\']:\n            candidates.extend(direction_students[direction.lower()])\n\n        for candidate in candidates:\n            candidate[\'avg_proj\'] = candidate[\'score\'] / avg  \n\n        # Сортировка кандидатов внутри данного направления по avg_proj (descending order)\n        candidates.sort(key=lambda x: x[\'avg_proj\'])\n\n        while project[\'team_size\'] > 0 and len(candidates) > 0:\n            student = candidates.pop(0)  # Получить лучшего студента (с наивысшим avg_proj)\n            project[\'team\'].append(student)\n            project[\'team_size\'] -= 1\n\n        # Пересчитать средний балл для команды внутри этого проекта\n        team_avg_score = avg_score

In [84]:
# v3

'''def distribute_students_3(directions, students, projects):
    direction_students = create_dict_of_students_by_direction(directions, students)

    assigned_students = set()  # Создать множество для отслеживания назначенных студентов

    for project in projects:
        project['team'] = []
        candidates = []

        for direction in project['direction']:
            candidates.extend(direction_students[direction.lower])

        # Проверка наличия студентов в кандидатах
        if not candidates:
            continue  # Если нет доступных студентов, перейти к следующему проекту

        # Определить средний рейтинг студентов в данном направлении
        direction_avg_score = avg_score([student for student in candidates if student not in assigned_students])

        # Сортировка кандидатов внутри данного направления по разнице между их рейтингом и средним рейтингом направления
        candidates.sort(key=lambda x: abs(x['score'] - direction_avg_score))

        while project['team_size'] > 0 and candidates:
            student = candidates.pop(0)

            project['team'].append(student)
            project['team_size'] -= 1
            assigned_students.add(student)

        # Вывести назначенных студентов для этого проекта
        print(f"Project: {project['name']}, {project['direction']}")
        print("Assigned Students:")
        for student in project['team']:
            print(f"- {student['name']}, {student['score']}")
        print()
'''

'def distribute_students_3(directions, students, projects):\n    direction_students = create_dict_of_students_by_direction(directions, students)\n\n    assigned_students = set()  # Создать множество для отслеживания назначенных студентов\n\n    for project in projects:\n        project[\'team\'] = []\n        candidates = []\n\n        for direction in project[\'direction\']:\n            candidates.extend(direction_students[direction.lower])\n\n        # Проверка наличия студентов в кандидатах\n        if not candidates:\n            continue  # Если нет доступных студентов, перейти к следующему проекту\n\n        # Определить средний рейтинг студентов в данном направлении\n        direction_avg_score = avg_score([student for student in candidates if student not in assigned_students])\n\n        # Сортировка кандидатов внутри данного направления по разнице между их рейтингом и средним рейтингом направления\n        candidates.sort(key=lambda x: abs(x[\'score\'] - direction_avg_score))

In [85]:
# v4  добавлен индикатор assigned для студентов, добавлено распределение студентов без направлений, в команду распределяются студенты с медианным score

'''def distribute_students_4(directions, students, projects):
    direction_students = create_dict_of_students_by_direction(directions, students)
    assigned_students = []  
    avg = avg_score(students)
    students_without_direction = [student for student in students if student['direction'] == '']


    for project in projects:
        project['team'] = []
        candidates = []
        team_size = project['team_size']

        for direction in project['direction']:
            candidates.extend(direction_students[direction.lower()])

        for candidate in candidates:
            candidate['avg_proj'] = candidate['score'] / avg
            candidate['assigned'] = False

        candidates.sort(key=lambda x: x['avg_proj'], reverse=True)

        while team_size > 0 and len(candidates) > 0:
            student = candidates.pop(len(candidates)//2)

            # Проверить, что студент не был уже назначен и не находится в списке assigned_students
            if not student['assigned'] and student not in assigned_students:
                project['team'].append(student)
                team_size -= 1
                student['assigned'] = True
                assigned_students.append(student)  # Добавить студента в список назначенных

        team_avg_score = avg_score(project['team'])

        while team_size > 0 and len(candidates) > 0:
            remaining_students = [student for student in candidates if
                                  student['direction'].lower() in project['direction'] and not student['assigned']]

            if remaining_students:
                remaining_students = [student for student in remaining_students if student not in assigned_students]
                remaining_students.sort(key=lambda x: abs(x['score'] - team_avg_score))
                student = remaining_students.pop(len(remaining_students)//2) #Выбирается медиана
                project['team'].append(student)
                team_size -= 1
                student['assigned'] = True
                assigned_students.append(student)

        while team_size > 0 and students_without_direction:
            student = students_without_direction.pop()
            project['team'].append(student)
            team_size -= 1
        if team_size > 0: project['status'] = 'incomplete'
        else: project['status'] = 'complete'

    for project in projects:
        print(f"Project: {project['name']}, {project['direction']}, {project['status']}")
        print("Assigned Students:")
        for student in project['team']:
            print(f"- {student['name']}, {student['score']}")
        print()
'''

'def distribute_students_4(directions, students, projects):\n    direction_students = create_dict_of_students_by_direction(directions, students)\n    assigned_students = []  \n    avg = avg_score(students)\n    students_without_direction = [student for student in students if student[\'direction\'] == \'\']\n\n\n    for project in projects:\n        project[\'team\'] = []\n        candidates = []\n        team_size = project[\'team_size\']\n\n        for direction in project[\'direction\']:\n            candidates.extend(direction_students[direction.lower()])\n\n        for candidate in candidates:\n            candidate[\'avg_proj\'] = candidate[\'score\'] / avg\n            candidate[\'assigned\'] = False\n\n        candidates.sort(key=lambda x: x[\'avg_proj\'], reverse=True)\n\n        while team_size > 0 and len(candidates) > 0:\n            student = candidates.pop(len(candidates)//2)\n\n            # Проверить, что студент не был уже назначен и не находится в списке assigned_stude

In [86]:
# v5 team_size одинаковый 

'''def distribute_students(directions, students, projects, team_size):
    direction_students = create_dict_of_students_by_direction(directions, students)
    assigned_students = []  
    avg = avg_score(students)
    students_without_direction = [student for student in students if len(student['directions']) == 0]

    for project in projects:
        size = team_size
        project['team'] = []
        candidates = []

        for direction in project['directions']:
            candidates.extend(direction_students[direction.lower()])

        for candidate in candidates:
            if ('assigned' in candidate.keys()) and (candidate['assigned'] == True):
               candidate['avg_proj'] = candidate['score'] / avg 
            else: 
               candidate['avg_proj'] = candidate['score'] / avg
               candidate['assigned'] = False

        candidates.sort(key=lambda x: x['avg_proj'])

        while size > 0 and len(candidates) > 0:
            student = candidates.pop(len(candidates)//2)

            # Проверить, что студент не был уже назначен и не находится в списке assigned_students
            if not student['assigned'] and student not in assigned_students:
                project['team'].append(student)
                size -= 1
                student['assigned'] = True
                assigned_students.append(student)  # Добавить студента в список назначенных

        team_avg_score = avg_score(project['team'])

        while size > 0 and len(candidates) > 0:
            remaining_students = [student for student in candidates if
                                  student['directions'].lower() in project['directions'] and not student['assigned']]

            if remaining_students:
                remaining_students = [student for student in remaining_students if student not in assigned_students]
                remaining_students.sort(key=lambda x: abs(x['score'] - team_avg_score))
                student = remaining_students.pop(len(remaining_students)//2) #Выбирается медиана
                project['team'].append(student)
                size -= 1
                student['assigned'] = True
                assigned_students.append(student)

        while size > 0 and len(students_without_direction) > 0:
            student = students_without_direction.pop()
            project['team'].append(student)
            size -= 1

            
        if size > 0: project['status'] = 'incomplete'
        else: project['status'] = 'complete'
    

    for project in projects:
        print(f"Project: {project['name']}, {project['directions']}, {project['status']}")
        print("Assigned Students:")
        for student in project['team']:
            print(f"- {student['st']}, {student['score']}")
        print()

    return students, projects    
'''

'def distribute_students(directions, students, projects, team_size):\n    direction_students = create_dict_of_students_by_direction(directions, students)\n    assigned_students = []  \n    avg = avg_score(students)\n    students_without_direction = [student for student in students if len(student[\'directions\']) == 0]\n\n    for project in projects:\n        size = team_size\n        project[\'team\'] = []\n        candidates = []\n\n        for direction in project[\'directions\']:\n            candidates.extend(direction_students[direction.lower()])\n\n        for candidate in candidates:\n            if (\'assigned\' in candidate.keys()) and (candidate[\'assigned\'] == True):\n               candidate[\'avg_proj\'] = candidate[\'score\'] / avg \n            else: \n               candidate[\'avg_proj\'] = candidate[\'score\'] / avg\n               candidate[\'assigned\'] = False\n\n        candidates.sort(key=lambda x: x[\'avg_proj\'])\n\n        while size > 0 and len(candidates) >

In [87]:
# Распределение по остаточному признаку

def distribute_remaining_students(students, projects, avg):
    assigned_students = []
    for student in students:
        if ('assigned' in student.keys()) and (student['assigned'] == True):
            student['avg_proj'] = student['score'] / avg 
        else: 
            student['avg_proj'] = student['score'] / avg
            student['assigned'] = False

    remaining_students = [student for student in students if ('assigned' in student.keys()) and (student['assigned'] == False)]
    remaining_students.sort(key=lambda x: x['avg_proj'])
    for project in projects:
        remaining_roles = project['remaining_roles']
        while len(remaining_students) > 0 and project['remainder'] > 0 and len(remaining_roles) > 0:
            for role in remaining_roles:
               if len(project[role]) == project['compound'].count(role):
                      remaining_roles.remove(role)
                      break
               
               student = remaining_students.pop(len(remaining_students)//2)
               project[role].append([student['st'], student['score'], student['year']])
               project['team'].append([student['st'], student['score'], student['year'], student['season']])
               project['remainder'] -= 1
               student['assigned'] = True
               assigned_students.append(student)  # Добавить студента в список назначенных
               remaining_roles.remove(role)
               break

    for project in projects:    
        while project['remainder'] > 0 and len(remaining_students) > 0:
            student = remaining_students.pop(len(remaining_students)//2)
            project['team'].append([student['st'], student['score'], student['year'], student['season']])
            project['remainder'] -= 1
            student['assigned'] = True
            assigned_students.append(student)            

        if project['remainder'] == 0: 
            project['status'] = 'complete'
        
        project['team_mean_score'] = np.mean([s[1] for s in project['team']])

    
    return students, projects
            
    

In [101]:
# v6

def distribute_students(directions, students, projects, team_size=False):
    
    direction_students = create_dict_of_students_by_direction(directions, students)
    assigned_students = []  
    avg = avg_score(students)
    students_without_direction = [student for student in students if len(student['directions']) == 0] 
    students_without_roles = [student for student in students if len(student['roles']) == 0]
    projects_distributed = projects.copy()
    if team_size:
        size = team_size
    else: 
#       size = len(project['compound'])
        size = len(students)//len(projects_distributed)
        if len(students)%len(projects_distributed) > 0:
            size += 1 
    
    for project in projects_distributed:
        s = size
        project['team'] = []
        remaining_roles = project['compound'].copy()
        candidates = []

        for direction in project['directions']:
            candidates.extend(direction_students[direction.lower()])

        for candidate in candidates:
            if ('assigned' in candidate.keys()) and (candidate['assigned'] == True):
               candidate['avg_proj'] = candidate['score'] / avg 
            else: 
               candidate['avg_proj'] = candidate['score'] / avg
               candidate['assigned'] = False

        candidates.sort(key=lambda x: x['avg_proj'])

        while s > 0 and len(candidates) > 0:
            student = candidates.pop(len(candidates)//2)
            # Проверить, что студент не был уже назначен и не находится в списке assigned_students
            if not student['assigned'] and student not in assigned_students:
               for role in remaining_roles:
                   if len(project[role]) == project['compound'].count(role):
                      remaining_roles.remove(role)
                      break
                   
                   if role in student['roles']:
                      project[role].append([student['st'], student['score'], student['year'], student['season']])
                      project['team'].append([student['st'], student['score'], student['year'], student['season']])
                      s -= 1
                      student['assigned'] = True
                      assigned_students.append(student)  # Добавить студента в список назначенных
                      remaining_roles.remove(role)
                      break

        '''team_avg_score = avg_score(project['team'])

      while size > 0 and len(candidates) > 0:
            candidates.sort(key=lambda x: abs(x['score'] - team_avg_score))
            student = candidates.pop(len(candidates)//2) #Выбирается медиана
            if not student['assigned'] and student not in assigned_students:
                project['team'].append(student)
                size -= 1
                student['assigned'] = True
                assigned_students.append(student)
'''
        while s > 0 and len(students_without_direction) > 0:
            student = students_without_direction.pop(len(students_without_direction)//2)
            for role in remaining_roles:
                   if len(project[role]) == project['compound'].count(role):
                       remaining_roles.remove(role)
                       break
                   
                   if (role in student['roles']):
                      project['team'].append([student['st'], student['score'], student['year'], student['season']])
                      project[role].append([student['st'], student['score'], student['year']])
                      remaining_roles.remove(role)
                      s -= 1
        
        while s > 0 and len(students_without_roles) > 0:
            student = students_without_roles.pop(len(students_without_roles)//2)
            for role in remaining_roles:
                    if len(project[role]) == project['compound'].count(role):
                       break
                   
                    project['team'].append([student['st'], student['score'], student['year'], student['season']])
                    project[role].append([student['st'], student['score'], student['year']])
                    remaining_roles.remove(role)
                    s -= 1

        if size > 0: 
            project['status'] = 'incomplete'
            project['remaining_roles'] = remaining_roles
            project['remainder'] = s
        else: 
            project['status'] = 'complete'
            project['remaining_roles'] = []
            project['remainder'] = 0

    students, projects_distributed = distribute_remaining_students(students, projects_distributed, avg)  


    for project in projects_distributed:
        print(f"Project: {project['name']}, {project['directions']}, {project['status']}, {project['team_mean_score']}, {project['compound']}, {len(project['team'])}")
        print("Assigned Students:")
        for p in set(project['compound']):
            print(f"- {p}: {project[p]}")
        print()

    return students, projects_distributed

In [102]:
students, projects = data_preparation(df, projects_df)
directions = get_directions_of_projects(projects)    
#team_size = 6
s,p = distribute_students(directions, students, projects)

Project: Разработать прототип игры с адаптивной сложностью прохождения, ['GameDev'], incomplete, 11.125, ['backend developer', 'frontend developer', 'ux/ui', 'qa', 'pm'], 4
Assigned Students:
- qa: [['ST087238', 10.9, 2023, 'Весна']]
- pm: [['ST087129', 11.2, 2022, 'Осень']]
- ux/ui: [['ST087396', 11.3, 2022, 'Осень']]
- backend developer: [['ST068916', 10.6, 2023, 'Осень']]
- frontend developer: [['ST049271', 11.1, 2020, 'Осень']]

Project: Разработать прототип игры с адаптивной сложностью прохождения, ['GameDev'], complete, 11.080000000000002, ['backend developer', 'frontend developer', 'ux/ui', 'qa', 'pm'], 5
Assigned Students:
- qa: [['ST068737', 11.7, 2020, 'Осень']]
- pm: [['ST115984', 11.0, 2023, 'Осень']]
- ux/ui: [['ST064150', 10.8, 2019, 'Осень']]
- backend developer: [['ST085077', 11.0, 2022, 'Осень']]
- frontend developer: [['ST068144', 10.9, 2023, 'Осень']]

Project: Голосовой бот для сбора данных для измерения NPS, ['Web', 'DS'], complete, 10.46, ['da or de', 'backend dev

c:\Users\Alyonka\micromamba\envs\auto-distribution-alyona-denv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Alyonka\micromamba\envs\auto-distribution-alyona-denv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [103]:
#Пример для 2023 года Осень

df_students_2023 = df[(df['Year'] == 2023) & (df['Season'] == 'Осень')]
df_projects_2023 = projects_df[(projects_df['Год'] == 2023) & (projects_df['Сезон'] == 'Осень')]

In [104]:
df_students_2022 = df[(df['Year'] == 2022) & (df['Season'] == 'Осень')]
df_projects_2022 = projects_df[(projects_df['Год'] == 2022) & (projects_df['Сезон'] == 'Осень')]

In [92]:
df_projects_2023

,Проект,Заказчик,Направление,Требования,Год,Сезон,Состав команды
0,Разработать прототип игры с адаптивной сложнос...,CloverСreek Ent,GameDev,NaN,2023,Осень,"backend developer:ST068916, frontend developer..."
1,Разработать прототип игры с адаптивной сложнос...,CloverСreek Ent,GameDev,NaN,2023,Осень,"backend developer, frontend developer,ux/ui, q..."
2,Голосовой бот для сбора данных для измерения NPS,TargetAI,"Web, DS",NaN,2023,Осень,"da or de, backend developer, system or busines..."
3,Геоинформационная система для точного земледелия,АгроФиз,Web,NaN,2023,Осень,"backend developer, frotend developer, ux/ui, q..."
4,Создание полевого робота для агроскаутинга (Аг...,АгроФиз,"Robotics, Sensors",NaN,2023,Осень,"backend developer, implementation, qa, da or d..."
5,Определение положения робота-манипулятора с по...,АО Цифровая сборка,"Robotics, DS",NaN,2023,Осень,"backend developer, implementation, qa, da or d..."
6,Модуль планирования расходных материалов,Виста,Web,NaN,2023,Осень,"backend developer, frotend developer, ux/ui, q..."
7,Личный кабинет пациента,Виста,Web,NaN,2023,Осень,"backend developer, frotend developer, ux/ui, q..."
8,Smart-клиника,Виста,Web,NaN,2023,Осень,"backend developer, frotend developer, ux/ui, q..."
9,Автоматизация компетентностного распределения ...,ИТ Клиника,DS,NaN,2023,Осень,"da or de, da or de, system or business analyst..."


In [107]:
students, projects = data_preparation(df_students_2022, df_projects_2022)
directions = get_directions_of_projects(projects)    
#team_size = 8
s_22,p_22 = distribute_students(directions, students, projects)

Project: I, II. Прямая и обратная позиционная задача. III. Уточнение параметров робота, ['Robotics'], complete, 10.0125, ['backend developer', 'backend developer', 'implementation', 'qa', 'pm'], 8
Assigned Students:
- qa: [['ST087408', 6.4, 2022]]
- implementation: [['ST087129', 11.2, 2022, 'Осень']]
- backend developer: [['ST091150 ', 10.6, 2022, 'Осень'], ['ST085954', 10.8, 2022, 'Осень']]
- pm: [['ST087340', 10.6, 2022, 'Осень']]

Project: Веб-сервис по цифровой диагностики сердца, ['Web'], complete, 11.1375, ['backend developer', 'frotend developer', 'ux/ui', 'qa', 'pm'], 8
Assigned Students:
- qa: [['ST086980', 10.1, 2022, 'Осень']]
- pm: [['ST096581', 13.7, 2022, 'Осень']]
- ux/ui: [['ST087725', 9.7, 2022, 'Осень']]
- backend developer: [['ST089430', 10.6, 2022, 'Осень']]
- frotend developer: [['ST055825', 9.8, 2022]]

Project: Агрегатор IT мероприятий, ['Web', 'DS'], complete, 10.850000000000001, ['da or de', 'backend developer', 'system or business analyst', 'frotend developer'

In [108]:
data_st = pd.DataFrame(s_22)
data_st.to_excel('scorred_students_22autumn.xlsx')
data_p = pd.DataFrame(p_22)
data_p.to_excel('distributed_projects_22autumn.xlsx')

In [109]:
students, projects = data_preparation(df_students_2023, df_projects_2023)
directions = get_directions_of_projects(projects)    
#team_size = 8
s_23,p_23 = distribute_students(directions, students, projects)

Project: Разработать прототип игры с адаптивной сложностью прохождения, ['GameDev'], incomplete, 13.049999999999999, ['backend developer', 'frontend developer', 'ux/ui', 'qa', 'pm'], 4
Assigned Students:
- qa: [['ST113210', 17.4, 2023]]
- pm: [['ST110105', 11.8, 2023, 'Осень']]
- ux/ui: [['ST110277', 10.4, 2023, 'Осень']]
- backend developer: [['ST068916', 10.6, 2023, 'Осень']]
- frontend developer: [['ST097831', 12.6, 2023, 'Осень']]

Project: Разработать прототип игры с адаптивной сложностью прохождения, ['GameDev'], incomplete, 11.260000000000002, ['backend developer', 'frontend developer', 'ux/ui', 'qa', 'pm'], 5
Assigned Students:
- qa: [['ST113273', 11.6, 2023]]
- pm: [['ST115984', 11.0, 2023, 'Осень']]
- ux/ui: [['ST105896', 9.7, 2023, 'Осень']]
- backend developer: [['ST115985', 12.6, 2023, 'Осень']]
- frontend developer: [['ST115987', 11.4, 2023, 'Осень']]

Project: Голосовой бот для сбора данных для измерения NPS, ['Web', 'DS'], complete, 11.883333333333333, ['da or de', 'bac

In [112]:
# Проверка, что не осталось не назначенных студентов

for s in s_23:
    if s['assigned'] == False:
        print(s)


In [110]:
data_st = pd.DataFrame(s_23)
data_st.to_excel('scorred_students_23autumn.xlsx')

In [111]:
data = pd.DataFrame(p_23)
data.to_excel('distributed_projects_23autumn.xlsx')

In [99]:
s_23

[{'st': 'ST110311',
  'year': 2023,
  'season': 'Осень',
  'score': 10.1,
  'directions': ['Web', 'NLP'],
  'roles': [],
  'avg_proj': 0.8459927660384543,
  'assigned': False},
 {'st': 'ST063118',
  'year': 2023,
  'season': 'Осень',
  'score': 13.7,
  'directions': ['Time Series', 'DS', 'Robotics'],
  'roles': ['backend developer'],
  'avg_proj': 1.1475347420521607,
  'assigned': False},
 {'st': 'ST114946',
  'year': 2023,
  'season': 'Осень',
  'score': 13.7,
  'directions': ['CV', 'DS'],
  'roles': ['frontend developer'],
  'avg_proj': 1.1475347420521607,
  'assigned': False},
 {'st': 'ST112047',
  'year': 2023,
  'season': 'Осень',
  'score': 10.8,
  'directions': ['Web', 'NLP', 'Robotics'],
  'roles': ['pm'],
  'avg_proj': 0.9046259280411195,
  'assigned': False},
 {'st': 'ST112471',
  'year': 2023,
  'season': 'Осень',
  'score': 11.3,
  'directions': [],
  'roles': ['pm'],
  'avg_proj': 0.9465067580430232,
  'assigned': False},
 {'st': 'ST069808',
  'year': 2023,
  'season': 'Ос

In [100]:
print('Введите название 1-го проекта и ST студента через ;')
answer1 = input('Название проекта; ST студента')
project1 = answer1.split(';')[0].strip()
student1 = answer1.split(';')[1].strip()

print('Введите название 2-го проекта и ST студента через ;')
answer2 = input('Название проекта, ST студента')
project2 = answer2.split(';')[0].strip()
student2 = answer2.split(';')[1].strip()

student_exchange(project1, student1, project2, student2, p_22)

Введите название 1-го проекта и ST студента через ;


IndexError: list index out of range

Team:

{'backend developer',
 'da or de',
 'frontend developer',
 'implementation',
 'pm',
 'qa',
 'system or business analyst',
 'ux/ui'}

['GameDev'] -- PM, ux/ui, backend developer, frontend developer, QA, 

['Web', 'DS'], ['DS', 'Web'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['Web'] -- PM, Backend, Frotend, UI/UX, QA

['Robotics', 'Sensors'] -- PM, implementation, qa, backend developer, da or de

['Robotics', 'DS'] -- PM, implementation, qa, backend developer, da or de

['DS'] -- PM, da or de, da or de, system or business analyst, backend developer 

['Data Analysis'] -- PM, da or de, da or de, system or business analyst, backend developer 

['DS', 'Data Analysis'] -- PM, da or de, da or de, system or business analyst, backend developer

['Robotics'] --  PM, implementation, qa, backend developer, backend developer 

['CV'] -- PM, da or de, da or de, system or business analyst, backend developer 

['Web', 'Data Analysis'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['Web', 'DS', 'Sensors'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['Web', 'Mobile Development'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['Robotics', 'CV'] -- PM, implementation, qa, backend developer, da or de

['Service Development'] -- PM, Backend, Frotend, UI/UX, QA

['Service Development', 'Data Analysis'] -- PM, Backend, Frotend, QA, da or de

['CV', 'Service Development'] -- PM, Backend, Frotend, UI/UX, QA, da or de

['NLP', 'Service Development'] -- PM, Backend, Frotend, UI/UX, QA, da or de

['NLP'] -- PM, da or de, da or de, system or business analyst, backend developer 

['Data Analysis', 'Web', 'IoT'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['NLP', 'Web'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['CV', 'NLP', 'Web'] -- PM, Backend, Frotend, da or de, QA, system or business analyst

['Mobile Development'] -- PM, Backend, Frotend, da or de, QA, system or business analyst, ux/ui